In [28]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [30]:
%run -i ../../core.py

In [31]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [32]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: make the bound approx same as real data
dummy_data = np.round(np.random.uniform(1,10,len(data)),1)
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
dummy_theory_output = torch.var(dummy_data_tensor, correction = 1)
dummy_data_mean = torch.mean(dummy_data_tensor)

In [33]:
# Verifier/ data consumer side:
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = dummy_data_mean, requires_grad = False)

    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*(X.size()[1]-1))<=torch.abs(0.01*self.w*(X.size()[1]-1)),x_mean_cons ),self.w)

verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

In [34]:
# Prover/ data owner side
theory_output = torch.var(data_tensor, correction = 1)
data_mean = torch.mean(data_tensor)
print("theory output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.data_mean = nn.Parameter(data = data_mean, requires_grad = False)
    def forward(self,X):
        x_mean_cons = torch.abs(torch.sum(X)-X.size()[1]*(self.data_mean))<=torch.abs(0.01*X.size()[1]*self.data_mean)
        return (torch.logical_and(torch.abs(torch.sum((X-self.data_mean)*(X-self.data_mean))-self.w*(X.size()[1]-1))<=torch.abs(0.01*self.w*(X.size()[1]-1)),x_mean_cons ),self.w)


prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path, "default", "resources", settings_path)

theory output:  tensor(212.5777)
==== Generate & Calibrate Setting ====
scale:  default
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-20332,8],"logrows":15,"num_inner_cols":1,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":1520,"total_const_size":8,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"5.0.8","num_blinding_factors":null}


In [35]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path, srs_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path, srs_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 4.619431257247925 seconds
Theory output:  tensor(212.5777)
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 212.5
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14958435514946940528, 17320992765071788231, 18318297740500540397, 2188255106962760522]]], 'proof': '145553cc45b28bf0c97a0731be6578b9a36c25979bc7bd2cb7daa9e6b3a3fc8c0bc9d47c2cf89b4670ee7ad5ad4eb368bbdeb363b9083fdc78f336ca5bcfe147136b7b4cc7bb1db75e1f0cdae26d9529ae2a22dfaa4e80ec448f9ff3f1a3638c0c5dd03286a8ef08f90afe6a3e21c386a1544ce80f256c1fce73c17d31e344d80e2c2f6c84ac5ac1b7162aef3f170e9a4b0b61d035e592386776a0d02b55b53304191c6d0cfc54c5a82f74fecacecbbc3206554f8fa9ff504572b1f35c020ce41c393d86bfd4c67cbe3345ae6a4e3820243914fc7256a897dd588004ab491fd718c5ceded40e8f1f579419af6d777123b4d555b1462847fe3ad55db4c7e04f6a113296f865f8cc2e5e40d69110bcd91427c3aa077d0adb94d944c004a12d7e571eb113472afeb21bef8475889a6b2176b9a6487bf8ebc2e18f043d41910998dc1b7318fb66ed238801571d1a2e9650f576a5e2b1df43750776f682ee8e59

In [36]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, srs_path)

num_inputs:  1
prf instances:  [[[12907834141446617622, 16863260785082668294, 2215826187815521673, 2191547160659437828], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [14958435514946940528, 17320992765071788231, 18318297740500540397, 2188255106962760522]]]
proof boolean:  1.0
proof result 1 : 212.5
verified
